In [1]:
import pandas as pd
from openpyxl import load_workbook
import os
import pandas as pd
import numpy as np

In [2]:
data_path = '..\\datasets'
files = os.listdir(data_path)
files.remove('URL descripcion reto.txt')
files.remove('leeme.txt')
dfs = {}

def open_file(file_name):
    file_path = os.path.join(data_path, file_name)

    # Cargar todas las hojas con pandas.read_excel(sheet_name=None)
    all_sheets = pd.read_excel(file_path, sheet_name=None)

    # Guardar cada hoja en dfs con un nombre único combinando el archivo y la hoja
    for sheet_name, data in all_sheets.items():
        # Crear un nombre único para el DataFrame usando el nombre del archivo y la hoja
        df_name = f"{file_name.replace('.xlsx', '')}_{sheet_name}"
        dfs[df_name] = data


# Abrir todos los archivos y leer todas las hojas
for file in files:
    if file.endswith('.xlsx'):  # Asegurarse de que solo se procesen archivos Excel
        open_file(file)
    if file.endswith('.XLSX'):
        open_file(file)


In [3]:
print(dfs.keys())

dict_keys(['Biorreactor 13169_WData', 'Biorreactor 13169_Datos', 'Biorreactor 13170_WData', 'Biorreactor 13170_Datos', 'Biorreactor 13171_WData', 'Biorreactor 13171_Datos', 'Biorreactor 13172_WData', 'Biorreactor 13172_Datos', 'Biorreactor 14614_Datos', 'Biorreactor 14615_WData', 'Biorreactor 14615_Datos', 'Biorreactor 14616_WData', 'Biorreactor 14616_Datos', 'Biorreactor 14617_WData', 'Biorreactor 14617_Datos', 'Biorreactor 14618_WData', 'Biorreactor 14618_Datos', 'Centrífuga 12912_WData', 'Centrífuga 12912_Datos', 'Centrífuga 14246_WData', 'Centrífuga 14246_Datos', 'Centrífuga 17825_WData', 'Centrífuga 17825_Datos', 'Cinéticos IPC_Inóculos', 'Cinéticos IPC_Cultivos finales', 'Cinéticos IPC_Centrifugación', 'Fases producción v02_Preinóculo', 'Fases producción v02_Inóculo', 'Fases producción v02_Cultivo final', 'Fases producción v03 Test_Cultivo final', 'Fases producción v03_Preinóculo', 'Fases producción v03_Inóculo', 'Fases producción v03_Cultivo final', 'Fases producción_test v02_Cu

In [4]:
# Acceder a los DataFrames

df_preinoculo = dfs['Fases producción v03_Preinóculo']
df_preinoculo.columns = df_preinoculo.iloc[0]
df_preinoculo = df_preinoculo.drop(0)
df_inoculo = dfs['Fases producción v03_Inóculo']
df_cultivo_final = dfs['Fases producción v03_Cultivo final']
df_of_123456 = dfs['OF 123456 v03_Sheet1'] 
df_of_123456 = df_of_123456.rename(columns={df_of_123456.columns[3]: 'LOTE'})
df_cineticos = dfs['Cinéticos IPC_Inóculos']
df_cineticos = df_cineticos.rename(columns={df_cineticos.columns[0]: 'LOTE'})
df_cineticos = df_cineticos[['LOTE', 'Turbidez', 'Viabilidad']].groupby('LOTE').mean()
df_of_123456['LOTE'] = df_of_123456['LOTE'].astype(str).str.replace('/', '').astype(int)

In [5]:
# Unir los tres DataFrames por la columna 'LOTE'
merged_df = df_preinoculo.merge(df_inoculo, on='LOTE', suffixes=('_preinoculo', '_inoculo'))
merged_df = merged_df.merge(df_of_123456[['LOTE', 'Cantidad entregada']], on='LOTE', suffixes=('', '_OF_123456'))

merged_df = merged_df.merge(df_cineticos, on='LOTE', suffixes=('', '_cineticos'))

# Unir los tres DataFrames por la columna 'LOTE'
merged_df = merged_df.merge(df_inoculo, on='LOTE', suffixes=('', '_inoculo'))
merged_df = merged_df.merge(df_cultivo_final, on='LOTE', suffixes=('', '_cultivo_final'))

In [6]:
import numpy as np

In [7]:
def load_centrifuga_data(id_centrifuga):
    file_name = f'Centrífuga {id_centrifuga}_Datos'
    try:
        # Intentar cargar el DataFrame de centrífuga
        centrifuga_df = dfs[file_name]
        return centrifuga_df
    except KeyError:
        # Si el archivo no existe, retornar un DataFrame vacío o NaN
        return pd.DataFrame()  # o simplemente return np.nan si prefieres


def calculate_mean_for_lote(lote, fecha_inicio, fecha_fin, id_centrifuga):
    centrifuga_df = load_centrifuga_data(id_centrifuga)
    if centrifuga_df.empty:
        return np.nan  # Si no hay datos, devolver NaN
    mask = (pd.to_datetime(centrifuga_df['DateTime']) >= pd.to_datetime(fecha_inicio)) & (pd.to_datetime(centrifuga_df['DateTime']) <= pd.to_datetime(fecha_fin))
    filtered_data = centrifuga_df[mask]
    return filtered_data[f'{id_centrifuga}_D01916047.PV'].mean()

merged_df['media_PV'] = merged_df.apply(
    lambda row: calculate_mean_for_lote(row['LOTE'], row['Fecha/hora inicio'], row['Fecha/hora fin'], row['ID Centrífuga']),
    axis=1
)

In [8]:
merged_df

,LOTE,Fecha/hora inicio_preinoculo,Fecha/hora fin_preinoculo,línea 1,línea 2,línea 3,línea 1,línea 2,línea 3,línea 1,...,Volumen de inóculo utilizado,Turbidez inicio cultivo_cultivo_final,Turbidez fin cultivo,Viabilidad final cultivo_cultivo_final,ID Centrífuga,Centrifugación 1 turbidez,Centrifugación 2 turbidez,Producto 1,Producto 2,media_PV
0,23019,2023-03-19 05:00:00,2023-03-20 07:24:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,82.40,17.28,91.20,184000000,17825,NaN,NaN,1747.920,6.00,0.330522
1,23020,2023-03-19 05:00:00,2023-03-20 07:24:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,80.40,18.80,91.20,181600000,14246,NaN,NaN,1676.160,6.56,0.426711
2,23021,2023-03-20 05:00:00,2023-03-21 06:37:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,66.40,16.16,86.40,248000000,17825,NaN,NaN,1928.496,8.08,0.409436
3,23022,2023-03-20 05:00:00,2023-03-21 06:37:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,85.60,18.48,83.20,229600000,12912,NaN,NaN,1782.800,5.92,0.080985
4,23023,2023-03-26 05:00:00,2023-03-27 07:21:00,5.496,5.504,5.52,28.32,27.92,32,1,...,77.60,17.12,74.40,132800000,17825,26.56,20.88,1861.840,2.96,0.368708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,24045,2024-03-10 00:30:00,2024-03-11 07:15:00,5.512,5.624,5.552,26.08,25.2,28.24,1,...,80.00,17.52,72.48,139200000,12912,27.84,17.76,1573.520,5.76,0.667548
127,24044,2024-03-14 00:30:00,2024-03-15 07:15:00,5.424,5.408,5.48,24.4,27.92,24.48,1,...,83.60,19.28,77.52,160800000,14246,30.72,20.68,1528.720,5.44,0.314792
128,24049,2024-03-14 00:30:00,2024-03-15 07:15:00,5.424,5.408,5.48,24.4,27.92,24.48,1,...,83.60,18.88,72.64,164800000,12912,30.56,17.00,1342.800,4.88,0.346299
129,24050,2024-03-21 00:30:00,2024-03-22 07:00:00,5.496,5.488,5.376,34,27.84,35.12,1,...,84.16,17.76,67.60,152000000,6379,29.44,26.64,1422.800,3.68,NaN


In [9]:
def calculate_mean_for_lote_temp(lote, fecha_inicio, fecha_fin):
    temp_df = dfs['Temperaturas y humedades_Datos']
    if temp_df.empty:
        return np.nan  # Si no hay datos, devolver NaN
    mask = (pd.to_datetime(temp_df['DateTime']) >= pd.to_datetime(fecha_inicio)) & (pd.to_datetime(temp_df['DateTime']) <= pd.to_datetime(fecha_fin))
    filtered_data = temp_df[mask]
    return filtered_data[['07781_TI1501.PV','07781_MI1501.PV']].mean()


merged_df[['temperature_preinoculo','humedad_preinoculo']] = merged_df.apply(
    lambda row: calculate_mean_for_lote_temp(row['LOTE'],row['Fecha/hora inicio_preinoculo'], row['Fecha/hora fin_preinoculo']), axis = 1
)

In [10]:



# Function to load bioreactor data based on the given ID
def load_bioreactor_data(id_bioreactor):
    file_name = f'Biorreactor {id_bioreactor}_Datos'
    try:
        # Attempt to load the DataFrame of the bioreactor
        bioreactor_df = dfs[file_name]
        return bioreactor_df
    except KeyError:
        # If the file does not exist, return an empty DataFrame
        return pd.DataFrame()  # Alternatively, return np.nan if preferred

# Function to calculate the mean for the specified lot and bioreactor
def calculate_mean_for_lote2(lote, fecha_inicio, fecha_fin, id_bioreactor):
    bioreactor_df = load_bioreactor_data(id_bioreactor)
    
    if bioreactor_df.empty:
        return np.nan, np.nan, np.nan  # Return NaN for all means if there's no data

    # Filter the data based on the specified date range
    mask = (pd.to_datetime(bioreactor_df['DateTime']) >= pd.to_datetime(fecha_inicio)) & \
           (pd.to_datetime(bioreactor_df['DateTime']) <= pd.to_datetime(fecha_fin))
    filtered_data = bioreactor_df[mask]
    
    # Calculate means for the specified columns
    mean_temp = filtered_data[f'{id_bioreactor}_FERM0101.Temperatura_PV'].mean() if not filtered_data.empty else np.nan
    mean_ph = filtered_data[f'{id_bioreactor}_FERM0101.Single_Use_pH_PV'].mean() if not filtered_data.empty else np.nan
    mean_do = filtered_data[f'{id_bioreactor}_FERM0101.Single_Use_DO_PV'].mean() if not filtered_data.empty else np.nan
    
    return mean_temp, mean_ph, mean_do

# Apply the function to each row of merged_df and store the results in new columns
merged_df[['media_temp_bioreactor', 'media_ph_biorreactor', 'media_PO_biorreactor']] = merged_df.apply(
    lambda row: calculate_mean_for_lote2(row['LOTE'], row['Fecha/hora inicio_inoculo'], row['Fecha/hora fin_inoculo'], row['ID bioreactor']),
    axis=1,
    result_type='expand'  # This allows unpacking into multiple columns
)

In [11]:
merged_df = merged_df.fillna("NA")
output_file_path = '../dataset_new.csv'  # Cambia esto a la ruta donde quieras guardar el archivo
merged_df.to_csv(output_file_path, index=False)

In [13]:
merged_df

,LOTE,Fecha/hora inicio_preinoculo,Fecha/hora fin_preinoculo,línea 1,línea 2,línea 3,línea 1,línea 2,línea 3,línea 1,...,Centrifugación 1 turbidez,Centrifugación 2 turbidez,Producto 1,Producto 2,media_PV,temperature_preinoculo,humedad_preinoculo,media_temp_bioreactor,media_ph_biorreactor,media_PO_biorreactor
0,23019,2023-03-19 05:00:00,2023-03-20 07:24:00,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,1747.920,6.00,0.330522,20.157241,50.339050,29.689535,6.191826,24.355140
1,23020,2023-03-19 05:00:00,2023-03-20 07:24:00,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,1676.160,6.56,0.426711,20.157241,50.339050,29.689535,6.191826,24.355140
2,23021,2023-03-20 05:00:00,2023-03-21 06:37:00,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,1928.496,8.08,0.409436,20.173527,52.099099,29.439842,6.028036,25.462923
3,23022,2023-03-20 05:00:00,2023-03-21 06:37:00,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,1782.800,5.92,0.080985,20.173527,52.099099,29.439842,6.028036,25.462923
4,23023,2023-03-26 05:00:00,2023-03-27 07:21:00,5.496,5.504,5.52,28.32,27.92,32,1,...,26.56,20.88,1861.840,2.96,0.368708,20.251146,48.900195,29.442474,6.002998,23.396945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,24050,2024-03-21 00:30:00,2024-03-22 07:00:00,5.496,5.488,5.376,34,27.84,35.12,1,...,29.44,26.64,1422.800,3.68,NA,20.238600,51.106078,29.520042,5.885699,25.183600
577,24050,2024-03-21 00:30:00,2024-03-22 07:00:00,5.496,5.488,5.376,34,27.84,35.12,1,...,29.44,26.64,1422.800,3.68,NA,20.238600,51.106078,29.520042,5.885699,25.183600
578,24051,2024-03-21 00:30:00,2024-03-22 07:00:00,5.496,5.488,5.376,34,27.84,35.12,1,...,33.44,19.32,1486.560,5.52,0.535963,20.238600,51.106078,29.520042,5.885699,25.183600
579,24051,2024-03-21 00:30:00,2024-03-22 07:00:00,5.496,5.488,5.376,34,27.84,35.12,1,...,33.44,19.32,1486.560,5.52,0.535963,20.238600,51.106078,29.520042,5.885699,25.183600
